In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import copy

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings
warnings.filterwarnings('ignore')


# 1. 多模态数据集类
class MultimodalDataset(Dataset):
    def __init__(self, ts_data=None, vector_data=None, targets=None, is_test=False, device=device):
        self.ts_data = ts_data  # (N, K, D)
        self.vector_data = vector_data  # (N, M)
        self.targets = targets  # 标签
        self.is_test = is_test
        self.device = device
        
        # 数据标准化
        if ts_data is not None:
            self.ts_mean = np.mean(ts_data, axis=(0, 1), keepdims=True)
            self.ts_std = np.std(ts_data, axis=(0, 1), keepdims=True) + 1e-8
            self.ts_data = (ts_data - self.ts_mean) / self.ts_std
            self.num_samples = len(ts_data)
        if vector_data is not None:
            self.vec_mean = np.mean(vector_data, axis=0, keepdims=True)
            self.vec_std = np.std(vector_data, axis=0, keepdims=True) + 1e-8
            self.vector_data = (vector_data - self.vec_mean) / self.vec_std
            if self.ts_data is None:
                self.num_samples = len(vector_data)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        sample = {'index': idx}
        if self.ts_data is not None:
            sample['ts'] = torch.tensor(self.ts_data[idx], dtype=torch.float32).to(self.device)
        if self.vector_data is not None:
            sample['vector'] = torch.tensor(self.vector_data[idx], dtype=torch.float32).to(self.device)
        if self.targets is not None:
            sample['target'] = torch.tensor(self.targets[idx], dtype=torch.float32).to(self.device)
        return sample


# 2. 增强版模态编码器
class TSEncoder(nn.Module):
    """增强版时间序列编码器：多尺度特征提取+残差连接"""
    def __init__(self, input_dim, hidden_dim=128, output_dim=128):
        super().__init__()
        # 多尺度卷积+LSTM融合
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        # 双向LSTM+残差连接
        self.lstm1 = nn.LSTM(
            input_size=128,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        self.lstm2 = nn.LSTM(
            input_size=hidden_dim*2,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        
        # 时间注意力+门控机制
        self.time_attn = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        self.gate = nn.Linear(hidden_dim*2 + 128, hidden_dim*2)  # 融合卷积与LSTM特征
        
        # 输出投影
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # x: (batch_size, K, D) -> 卷积需要 (batch, D, K)
        conv_in = x.permute(0, 2, 1)  # 维度转换
        conv1_out = self.conv1(conv_in)  # (batch, 64, K)
        conv2_out = self.conv2(conv1_out)  # (batch, 128, K)
        conv_feat = conv2_out.permute(0, 2, 1)  # (batch, K, 128)
        
        # LSTM特征提取
        lstm1_out, _ = self.lstm1(conv_feat)  # (batch, K, 2*hidden)
        lstm2_out, _ = self.lstm2(lstm1_out + conv_feat)  # 残差连接
        
        # 时间注意力
        attn_weights = F.softmax(self.time_attn(lstm2_out).squeeze(-1), dim=1)  # (batch, K)
        attn_feat = torch.bmm(attn_weights.unsqueeze(1), lstm2_out).squeeze(1)  # (batch, 2*hidden)
        
        # 门控融合卷积全局特征与注意力特征
        global_conv_feat = torch.mean(conv_feat, dim=1)  # (batch, 128)
        gate_input = torch.cat([attn_feat, global_conv_feat], dim=1)
        gate = torch.sigmoid(self.gate(gate_input))
        fused = gate * attn_feat + (1 - gate) * global_conv_feat
        
        return self.fc(self.dropout(fused))


class VectorEncoder(nn.Module):
    """增强版向量编码器：深度MLP+特征交互"""
    def __init__(self, input_dim, output_dim=128):
        super().__init__()
        # 深度MLP+残差
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        # 特征交互层（捕捉特征间非线性关系）
        self.interact = nn.Sequential(
            nn.Linear(128, 128),
            nn.Tanh()
        )
        
        # 输出层
        self.fc = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # 残差连接
        out = self.layer1(x)
        residual = out
        out = self.layer2(out)
        out = self.layer3(out) + residual  # 残差连接
        
        # 特征交互
        interacted = self.interact(out) * out  # 元素级交互
        return self.fc(self.dropout(interacted))


# 3. 增强版多模态融合模型
class MultimodalEncoder(nn.Module):
    def __init__(self, ts_encoder, vector_encoder, latent_dim=128, output_dim=1):
        super().__init__()
        self.ts_encoder = ts_encoder
        self.vector_encoder = vector_encoder
        self.latent_dim = latent_dim
        
        # 动态模态注意力（替代固定权重）
        self.modal_attn = nn.Sequential(
            nn.Linear(128*2, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.Softmax(dim=1)
        )
        
        # 交叉注意力（双向交互）
        self.ts2vec_attn = nn.MultiheadAttention(embed_dim=128, num_heads=4, batch_first=True)
        self.vec2ts_attn = nn.MultiheadAttention(embed_dim=128, num_heads=4, batch_first=True)
        
        # 多尺度融合网络
        self.fusion = nn.Sequential(
            nn.Linear(128*2, 512),  # 更深的融合层
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.BatchNorm1d(latent_dim)
        )
        
        # 改进的预测头（带注意力）
        self.regressor = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        
        # 特征自注意力（增强全局依赖）
        self.self_attn = nn.MultiheadAttention(embed_dim=latent_dim, num_heads=4, batch_first=True)
        
        # 辅助投影
        self.projection = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

    def forward(self, x: dict, return_embedding=False):
        batch_size = next(iter(v for k, v in x.items() if k != 'index')).size(0)
        
        # 提取基础特征
        ts_feat = self.ts_encoder(x['ts']) if 'ts' in x else torch.zeros(batch_size, 128).to(device)
        vec_feat = self.vector_encoder(x['vector']) if 'vector' in x else torch.zeros(batch_size, 128).to(device)
        
        # 双向交叉注意力（模态间深度交互）
        ts_expand = ts_feat.unsqueeze(1)  # (B,1,128)
        vec_expand = vec_feat.unsqueeze(1)
        
        vec_attended_by_ts, _ = self.ts2vec_attn(vec_expand, ts_expand, ts_expand)  # 向量参考时间序列
        ts_attended_by_vec, _ = self.vec2ts_attn(ts_expand, vec_expand, vec_expand)  # 时间序列参考向量
        
        vec_attended = vec_attended_by_ts.squeeze(1)  # (B,128)
        ts_attended = ts_attended_by_vec.squeeze(1)   # (B,128)
        
        # 动态模态权重（基于当前样本特征自适应）
        modal_cat = torch.cat([ts_attended, vec_attended], dim=1)  # (B,256)
        weights = self.modal_attn(modal_cat)  # (B,2)
        ts_weighted = ts_attended * weights[:, 0:1]  # (B,128)
        vec_weighted = vec_attended * weights[:, 1:2]  # (B,128)
        
        # 融合特征
        fused = torch.cat([ts_weighted, vec_weighted], dim=1)  # (B,256)
        unified_embedding = F.relu(self.fusion(fused))  # (B, latent_dim)
        
        # 自注意力增强全局特征
        attn_emb, _ = self.self_attn(
            unified_embedding.unsqueeze(1),
            unified_embedding.unsqueeze(1),
            unified_embedding.unsqueeze(1)
        )
        unified_embedding = attn_emb.squeeze(1) + unified_embedding  # 残差
        
        # 预测输出
        output = self.regressor(unified_embedding)
        projection = self.projection(unified_embedding)
        
        if return_embedding:
            return output, projection, unified_embedding
        return output, projection


# 4. 增强版联邦客户端（修复对比损失批次不匹配问题）
class Client:
    def __init__(self, client_id, model, train_dataset, test_dataset=None, learning_rate=0.0005):
        self.client_id = client_id
        self.model = copy.deepcopy(model).to(device)
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        self.train_feature_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
        self.test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False) if test_dataset else None
        
        # 优化器改进：带动量的AdamW
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=1e-5,  # L2正则化
            betas=(0.9, 0.999)
        )
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(  # 学习率调度
            self.optimizer, mode='min', factor=0.5, patience=2, verbose=False
        )
        self.local_epochs = 5  # 增加本地训练轮次
        
        # 改进的互信息映射（更深层）
        self.f_map = nn.Sequential(
            nn.Linear(model.latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ).to(device)
        for param in self.f_map.parameters():
            param.requires_grad = False

    # 保留核心损失函数，优化数值稳定性
    def compute_mi_loss_Y_Z(self, Z, Y):
        f_Z = self.f_map(Z)
        f_Z_norm = (f_Z - f_Z.mean(dim=0)) / (f_Z.std(dim=0) + 1e-8)
        Y_norm = (Y - Y.mean(dim=0)) / (Y.std(dim=0) + 1e-8)
        cov = torch.cov(torch.cat([f_Z_norm, Y_norm], dim=1).T)[0, 1]
        corr = cov / (torch.std(f_Z_norm) * torch.std(Y_norm) + 1e-8)
        return -torch.log(torch.clamp(torch.abs(corr), min=1e-6))  # 避免log(0)

    def compute_kl_loss_modal(self, modal_feats):
        if len(modal_feats) < 2:
            return 0.0
        kl_total = 0.0
        num_pairs = 0
        var = nn.Parameter(torch.tensor(0.1)).to(device)
        dist_list = [torch.distributions.Normal(feat.mean(dim=0), var) for feat in modal_feats]  # 更稳定的分布估计
        
        for i in range(len(dist_list)):
            for j in range(i+1, len(dist_list)):
                kl_ij = torch.distributions.kl.kl_divergence(dist_list[i], dist_list[j]).mean()
                kl_ji = torch.distributions.kl.kl_divergence(dist_list[j], dist_list[i]).mean()
                kl_total += (kl_ij + kl_ji) / 2
                num_pairs += 1
        return kl_total / (num_pairs + 1e-8)

    def compute_contrastive_loss_anti_forget(self, current_Z, prev_global_Z, history_global_Zs, temperature=0.3):
        """修复批次大小不匹配问题：确保正负样本与当前批次大小一致"""
        batch_size = current_Z.size(0)  # 获取当前批次的实际大小
        current_Z_norm = F.normalize(current_Z, dim=1)
        
        # 调整正样本大小与当前批次一致
        prev_global_Z_norm = F.normalize(prev_global_Z, dim=1)
        if prev_global_Z_norm.size(0) > batch_size:
            pos_samples = prev_global_Z_norm[:batch_size]  # 截断到当前批次大小
        elif prev_global_Z_norm.size(0) < batch_size:
            # 不足时重复填充（保持数据分布）
            repeat_times = (batch_size // prev_global_Z_norm.size(0)) + 1
            pos_samples = torch.repeat_interleave(prev_global_Z_norm, repeat_times, dim=0)[:batch_size]
        else:
            pos_samples = prev_global_Z_norm
        
        # 调整负样本大小与当前批次一致
        if history_global_Zs:
            neg_candidates = torch.cat([F.normalize(emb, dim=1) for emb in history_global_Zs], dim=0)
            # 过滤高相似度负样本
            sim_matrix = torch.matmul(current_Z_norm, neg_candidates.T)
            mask = sim_matrix < 0.5  # 只保留相似度<0.5的负样本
            
            neg_samples = []
            for i in range(batch_size):
                neg_i = neg_candidates[mask[i]]
                if neg_i.size(0) == 0:
                    neg_i = neg_candidates[torch.randperm(neg_candidates.size(0))[:1]]  # 兜底
                # 确保每个样本有一个负样本
                neg_samples.append(neg_i[0:1])  # 取第一个符合条件的负样本
            neg_samples = torch.cat(neg_samples, dim=0)  # 形状：(batch_size, latent_dim)
        else:
            # 无历史时，使用当前样本的随机排列作为负样本
            neg_samples = current_Z_norm[torch.randperm(batch_size)]
        
        # 计算相似度并拼接（此时批次大小已一致）
        pos_sim = torch.matmul(current_Z_norm, pos_samples.T).diag() / temperature  # (batch_size,)
        neg_sim = torch.matmul(current_Z_norm, neg_samples.T).diag() / temperature  # (batch_size,)
        logits = torch.stack([pos_sim, neg_sim], dim=1)  # (batch_size, 2)
        labels = torch.zeros(logits.size(0), dtype=torch.long).to(device)  # 正样本标签为0
        return F.cross_entropy(logits, labels)

    def local_train(self, global_model, prev_global_Z, history_global_Zs, mu=1e-3):
        self.model.load_state_dict(global_model.state_dict())
        self.model.train()
        best_loss = float('inf')
        
        for epoch in range(self.local_epochs):
            total_loss = 0.0
            for batch in self.train_dataloader:
                current_pred, _, current_Z = self.model(batch, return_embedding=True)
                
                # 收集模态特征
                modal_feats = []
                if 'ts' in batch:
                    modal_feats.append(self.model.ts_encoder(batch['ts']))
                if 'vector' in batch:
                    modal_feats.append(self.model.vector_encoder(batch['vector']))
                
                # 损失组合（动态权重）
                reg_loss = F.mse_loss(current_pred.squeeze(), batch['target'])
                mi_loss = self.compute_mi_loss_Y_Z(current_Z, batch['target'].unsqueeze(1))
                kl_loss = self.compute_kl_loss_modal(modal_feats) if modal_feats else 0
                contrast_loss = self.compute_contrastive_loss_anti_forget(
                    current_Z, prev_global_Z, history_global_Zs
                )
                
                # 动态调整损失权重（根据训练阶段）
                if epoch < self.local_epochs // 2:  # 前期侧重拟合
                    loss = reg_loss + mu * (mi_loss + 0.5*kl_loss + 0.5*contrast_loss)
                else:  # 后期侧重泛化
                    loss = reg_loss + mu * (0.5*mi_loss + kl_loss + contrast_loss)
                
                self.optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)  # 梯度裁剪防爆炸
                self.optimizer.step()
                total_loss += loss.item()
            
            avg_loss = total_loss / len(self.train_dataloader)
            self.scheduler.step(avg_loss)  # 学习率调度
            if avg_loss < best_loss:
                best_loss = avg_loss  # 记录最优损失
        
        print(f"客户端 {self.client_id} 最佳平均损失: {best_loss:.4f}")
        return self.model.state_dict()

    # 特征提取接口
    def extract_train_features(self, use_global_model=False, global_model=None):
        model = global_model if (use_global_model and global_model) else self.model
        model.eval()
        all_features = []
        all_indices = []
        all_targets = []
        
        with torch.no_grad():
            for batch in self.train_feature_dataloader:
                _, _, emb = model(batch, return_embedding=True)
                all_features.append(emb.cpu())
                all_indices.extend(batch['index'].cpu().numpy())
                all_targets.extend(batch['target'].cpu().numpy())
        
        model.train()
        features_np = torch.cat(all_features, dim=0).numpy()
        targets_np = np.array(all_targets)
        
        return {
            'client_id': self.client_id,
            'train_indices': all_indices,
            'train_features': features_np,
            'train_targets': targets_np
        }

    def extract_test_features(self, use_global_model=False, global_model=None):
        if not self.test_dataset:
            return None
        
        model = global_model if (use_global_model and global_model) else self.model
        model.eval()
        all_features = []
        all_indices = []
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for batch in self.test_dataloader:
                pred, _, emb = model(batch, return_embedding=True)
                all_features.append(emb.cpu())
                all_indices.extend(batch['index'].cpu().numpy())
                all_preds.append(pred.squeeze().cpu().numpy())
                if 'target' in batch:
                    all_targets.extend(batch['target'].cpu().numpy())
        
        model.train()
        features_np = torch.cat(all_features, dim=0).numpy()
        preds_np = np.concatenate(all_preds, axis=0)
        targets_np = np.array(all_targets) if all_targets else None
        
        return {
            'client_id': self.client_id,
            'test_indices': all_indices,
            'test_features': features_np,
            'test_preds': preds_np,
            'test_targets': targets_np
        }


# 5. 增强版服务器
class Server:
    def __init__(self, model, num_clients):
        self.global_model = copy.deepcopy(model).to(device)
        self.num_clients = num_clients
        self.clients = []
        self.history_global_embs = []
        self.latent_dim = model.latent_dim
        self.client_weights = [1.0/num_clients for _ in range(num_clients)]  # 客户端权重

    def add_client(self, client):
        self.clients.append(client)

    def aggregate_parameters(self, client_params_list):
        """加权聚合（修复num_batches_tracked的类型错误）"""
        aggregated_params = {
            name: torch.zeros_like(param) 
            for name, param in self.global_model.state_dict().items()
        }
        
        for i, params in enumerate(client_params_list):
            w = self.client_weights[i]  # 客户端权重
            for name, param in params.items():
                if 'num_batches_tracked' in name:
                    # 整数型参数特殊处理：先按权重比例累加整数，再归一化
                    int_weight = int(round(w * self.num_clients))
                    aggregated_params[name] += param.long() * int_weight
                else:
                    # 浮点型参数正常加权
                    aggregated_params[name] += param.to(aggregated_params[name].dtype) * w
        
        # 最终处理：整数型参数归一化，浮点型参数平滑
        for name in aggregated_params:
            if 'num_batches_tracked' in name:
                aggregated_params[name] = (aggregated_params[name] // self.num_clients).long()
            else:
                aggregated_params[name] = 0.9 * aggregated_params[name] + 0.1 * self.global_model.state_dict()[name]
        
        self.global_model.load_state_dict(aggregated_params)
        return self.global_model.state_dict()

    def evaluate(self, test_dataset):
        if test_dataset is None:
            return
        self.global_model.eval()
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        total_mse = 0.0
        total_samples = 0
        
        with torch.no_grad():
            for batch in test_loader:
                pred, _ = self.global_model(batch)
                mse = F.mse_loss(pred.squeeze(), batch['target'], reduction='sum')
                total_mse += mse.item()
                total_samples += batch['target'].size(0)
        
        rmse = np.sqrt(total_mse / total_samples)
        #print(f"全局模型测试RMSE: {rmse:.4f}")
        self.global_model.train()
        return rmse

    def federated_train(self, rounds=10, global_test_dataset=None):
        for round_idx in range(rounds):
            print(f"\n===== 联邦轮次 {round_idx + 1}/{rounds} =====")
            
            client_params_list = []
            test_batch = None  # 初始化test_batch
            
            # 获取上一轮全局表征
            if global_test_dataset:
                test_loader = DataLoader(global_test_dataset, batch_size=32, shuffle=False)
                test_batch = next(iter(test_loader))
                
                if self.history_global_embs:
                    with torch.no_grad():
                        _, _, prev_global_Z = self.global_model(test_batch, return_embedding=True)
                    prev_global_Z = prev_global_Z.detach()
                    history_global_Zs = [z.detach() for z in self.history_global_embs[-5:]]
                else:
                    prev_global_Z = torch.zeros(32, self.latent_dim).to(device)
                    history_global_Zs = []
            else:
                prev_global_Z = torch.zeros(32, self.latent_dim).to(device)
                history_global_Zs = []
            
            # 客户端本地训练
            client_losses = []
            for client in self.clients:
                client_params = client.local_train(
                    self.global_model, prev_global_Z, history_global_Zs
                )
                client_params_list.append(client_params)
                client_losses.append(1.0)  # 实际应用中可替换为客户端返回的损失
            
            # 动态调整客户端权重
            if round_idx > 0:
                inv_loss = [1.0 / (loss + 1e-8) for loss in client_losses]
                self.client_weights = [w / sum(inv_loss) for w in inv_loss]
            
            # 聚合参数
            self.aggregate_parameters(client_params_list)
            
            # 保存全局表征
            if global_test_dataset and test_batch is not None:
                with torch.no_grad():
                    _, _, global_Z = self.global_model(test_batch, return_embedding=True)
                self.history_global_embs.append(global_Z.cpu())
                if len(self.history_global_embs) > 10:
                    self.history_global_embs.pop(0)
            
            # 评估全局模型
            self.evaluate(global_test_dataset)
        
        return self.global_model

    def get_all_client_features(self, use_global_model=True):
        all_train_features = []
        all_test_features = []
        
        for client in self.clients:
            train_feats = client.extract_train_features(
                use_global_model=use_global_model,
                global_model=self.global_model if use_global_model else None
            )
            all_train_features.append(train_feats)
            
            test_feats = client.extract_test_features(
                use_global_model=use_global_model,
                global_model=self.global_model if use_global_model else None
            )
            if test_feats:
                all_test_features.append(test_feats)
        
        return all_train_features, all_test_features

w:\Anaconda3\envs\DeepSumMoon\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# <center>**实际数据分析**<center>

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("./corn1.csv").iloc[:,1:]
X = np.array(data.iloc[:,:-4]).reshape(len(data),699,1)
Z = np.array(data.iloc[:,-4:-1])
y = np.array(data.iloc[:,-1])
train_ts, global_test_ts, train_vector,global_test_vector,train_target, global_test_target = train_test_split(X,Z,y, test_size=0.2)

In [16]:
print(f"训练集样本数：{train_vector.shape[0]}")  # 输出 64
print(f"测试集样本数：{global_test_ts.shape[0]}")    # 输出 16

训练集样本数：64
测试集样本数：16


In [17]:
# ----------------------------
# 2. 分配客户端数据（两模态）
# ----------------------------
client_train_datasets = []
client_test_datasets = []
for i in range(3):
    # 训练集
    start = i * 21
    end = start + 21
    train_data = MultimodalDataset(
        ts_data=train_ts[start:end],
        vector_data=train_vector[start:end],
        targets=train_target[start:end],
        is_test=False
    )
    client_train_datasets.append(train_data)

    # 测试集
    test_start = i * 5
    test_end = test_start + 5
    test_data = MultimodalDataset(
        ts_data=global_test_ts[test_start:test_end],
        vector_data=global_test_vector[test_start:test_end],
        targets=global_test_target[test_start:test_end],
        is_test=True
    )
    client_test_datasets.append(test_data)
    print(f"客户端 {i+1} - 训练样本: {len(train_data)}, 测试样本: {len(test_data)}")


# 全局测试集
global_test_dataset = MultimodalDataset(
    ts_data=global_test_ts,
    vector_data=global_test_vector,
    targets=global_test_target,
    is_test=True
)

客户端 1 - 训练样本: 21, 测试样本: 5
客户端 2 - 训练样本: 21, 测试样本: 5
客户端 3 - 训练样本: 21, 测试样本: 5


In [19]:
# ----------------------------
# 3. 初始化两模态联邦模型并训练
# ----------------------------
# 初始化编码器
ts_encoder = TSEncoder(
    input_dim=1,    # 时间序列每步特征维度
    hidden_dim=64,
    output_dim=128
)
vector_encoder = VectorEncoder(
    input_dim=3,   # 向量特征维度
    output_dim=128
)

# 初始化多模态融合模型
base_model = MultimodalEncoder(
    ts_encoder=ts_encoder,
    vector_encoder=vector_encoder,
    latent_dim=128,
    output_dim=1  # 回归任务
)

# 初始化服务器和客户端
server = Server(base_model, num_clients=3)
for i in range(3):
    client = Client(
        client_id=i+1,
        model=base_model,
        train_dataset=client_train_datasets[i],
        test_dataset=client_test_datasets[i]
    )
    server.add_client(client)

# 联邦训练
print("\n开始联邦训练...")
trained_global_model = server.federated_train(rounds=5, global_test_dataset=global_test_dataset)

# 提取所有客户端的特征（使用全局模型）
print("\n提取客户端特征...")
all_train_feats, all_test_feats = server.get_all_client_features(use_global_model=True)


开始联邦训练...

===== 联邦轮次 1/15 =====
客户端 1 最佳平均损失: 0.1136
客户端 2 最佳平均损失: 0.0961
客户端 3 最佳平均损失: 0.1056

===== 联邦轮次 2/15 =====
客户端 1 最佳平均损失: 0.0279
客户端 2 最佳平均损失: 0.0135
客户端 3 最佳平均损失: 0.0175

===== 联邦轮次 3/15 =====
客户端 1 最佳平均损失: 0.0244
客户端 2 最佳平均损失: 0.0171
客户端 3 最佳平均损失: 0.0232

===== 联邦轮次 4/15 =====
客户端 1 最佳平均损失: 0.0158
客户端 2 最佳平均损失: 0.0127
客户端 3 最佳平均损失: 0.0145

===== 联邦轮次 5/15 =====
客户端 1 最佳平均损失: 0.0107
客户端 2 最佳平均损失: 0.0101
客户端 3 最佳平均损失: 0.0143

===== 联邦轮次 6/15 =====
客户端 1 最佳平均损失: 0.0118
客户端 2 最佳平均损失: 0.0104
客户端 3 最佳平均损失: 0.0129

===== 联邦轮次 7/15 =====
客户端 1 最佳平均损失: 0.0121
客户端 2 最佳平均损失: 0.0080
客户端 3 最佳平均损失: 0.0107

===== 联邦轮次 8/15 =====
客户端 1 最佳平均损失: 0.0133
客户端 2 最佳平均损失: 0.0085
客户端 3 最佳平均损失: 0.0120

===== 联邦轮次 9/15 =====
客户端 1 最佳平均损失: 0.0099
客户端 2 最佳平均损失: 0.0081
客户端 3 最佳平均损失: 0.0130

===== 联邦轮次 10/15 =====
客户端 1 最佳平均损失: 0.0121
客户端 2 最佳平均损失: 0.0093
客户端 3 最佳平均损失: 0.0128

===== 联邦轮次 11/15 =====
客户端 1 最佳平均损失: 0.0145
客户端 2 最佳平均损失: 0.0107
客户端 3 最佳平均损失: 0.0097

===== 联邦轮次 12/15 =====
客户端 1 最佳平均损失: 0.0

In [20]:
# ----------------------------
# 4. 联邦特征的模型评估
# ----------------------------
# 存储各客户端的评估结果
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
results = []

# 遍历每个客户端进行模型训练与评估
print("\n===== 联邦特征的模型评估 =====")
for i in range(len(all_train_feats)):
    # 获取当前客户端的训练数据和测试数据
    train_data = all_train_feats[i]
    test_data = all_test_feats[i]
    client_id = train_data['client_id']
    
    print(f"处理客户端 {client_id}...")
    
    # 提取训练特征和标签
    X_train = train_data['train_features']
    y_train = train_data['train_targets']
    
    # 提取测试特征和真实标签
    X_test = test_data['test_features']
    y_test = test_data['test_targets']
    
    # 标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)  # 注意测试集用训练集的scaler
    
    # 训练随机森林模型
    rf_model = RandomForestRegressor(
        n_estimators=100,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    
    # 预测与评估
    y_pred = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # 存储结果
    results.append({
        '客户端ID': client_id,
        '训练样本数': len(train_data['train_indices']),
        '测试样本数': len(test_data['test_indices']),
        '特征维度': X_train.shape[1],
        '联邦特征-MSE': round(mse, 6)
    })
    
    # 保存预测结果
    test_data['test_preds'] = y_pred

# 汇总结果
results_df1 = pd.DataFrame(results)
print("\n===== 联邦特征评估结果 =====")
print(results_df1)


===== 联邦特征的模型评估 =====
处理客户端 1...
处理客户端 2...
处理客户端 3...

===== 联邦特征评估结果 =====
   客户端ID  训练样本数  测试样本数  特征维度  联邦特征-MSE
0      1     21      5   128  0.053679
1      2     21      5   128  0.025789
2      3     21      5   128  0.036576


In [21]:
# ===== 两模态：PCA / TSVD / RP + OURS（联邦特征）对比，一次性跑完并汇总 =====
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.random_projection import GaussianRandomProjection

# ----------------------------
# 5. 两模态特征的PCA降维与评估（保持你的原实现不变）
# ----------------------------
def pca_multimodal_data_by_contribution(ts_data, vector_data, contribution=0.9):
    """对时间序列和向量特征分别做PCA降维"""
    scaler = StandardScaler()
    modal_dims = {}  # 记录各模态降维后的维度
    
    # 1) 时间序列特征 PCA（先展平时间步）
    flat_ts = ts_data.reshape(ts_data.shape[0], -1)  # (N, K*D)
    ts_scaled = scaler.fit_transform(flat_ts)
    pca_ts = PCA().fit(ts_scaled)
    ts_cum_contrib = np.cumsum(pca_ts.explained_variance_ratio_)
    ts_dim = np.argmax(ts_cum_contrib >= contribution) + 1
    ts_dim = min(ts_dim, flat_ts.shape[1])  # 避免超过原始维度
    pca_ts = PCA(n_components=ts_dim)
    ts_pca = pca_ts.fit_transform(ts_scaled)
    modal_dims['ts'] = ts_dim
    modal_dims['ts_contrib'] = ts_cum_contrib[ts_dim-1]
    
    # 2) 向量特征 PCA
    vector_scaled = scaler.fit_transform(vector_data)
    pca_vector = PCA().fit(vector_scaled)
    vector_cum_contrib = np.cumsum(pca_vector.explained_variance_ratio_)
    vector_dim = np.argmax(vector_cum_contrib >= contribution) + 1
    vector_dim = min(vector_dim, vector_data.shape[1])
    pca_vector = PCA(n_components=vector_dim)
    vector_pca = pca_vector.fit_transform(vector_scaled)
    modal_dims['vector'] = vector_dim
    modal_dims['vector_contrib'] = vector_cum_contrib[vector_dim-1]
    
    # 总维度和平均贡献率
    modal_dims['total'] = ts_dim + vector_dim
    modal_dims['total_contrib'] = (ts_dim * modal_dims['ts_contrib'] + 
                                  vector_dim * modal_dims['vector_contrib']) / modal_dims['total']
    
    # 拼接所有模态的PCA特征
    return np.hstack([ts_pca, vector_pca]), modal_dims

def train_rf_with_pca_contribution(train_data, test_data, client_id, contribution=0.9):
    # 训练集 PCA 降维（按贡献率）
    X_train, modal_dims = pca_multimodal_data_by_contribution(
        train_data.ts_data,  # 时间序列
        train_data.vector_data,  # 向量
        contribution=contribution
    )
    y_train = train_data.targets
    
    # 测试集 PCA 降维（复用训练集的 PCA 参数）
    scaler = StandardScaler()
    # 时间序列
    flat_ts_test = test_data.ts_data.reshape(test_data.ts_data.shape[0], -1)
    ts_scaled_train = scaler.fit_transform(train_data.ts_data.reshape(train_data.ts_data.shape[0], -1))
    pca_ts = PCA(n_components=modal_dims['ts']).fit(ts_scaled_train)
    ts_pca_test = pca_ts.transform(scaler.transform(flat_ts_test))
    # 向量
    vector_scaled_train = scaler.fit_transform(train_data.vector_data)
    pca_vector = PCA(n_components=modal_dims['vector']).fit(vector_scaled_train)
    vector_pca_test = pca_vector.transform(scaler.transform(test_data.vector_data))
    
    X_test = np.hstack([ts_pca_test, vector_pca_test])
    y_test = test_data.targets
    
    # 随机森林（与您两模态代码保持一致）
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # 打印各模态降维信息
    print(f"客户端 {client_id} - PCA后总特征数: {modal_dims['total']}")
    print(f"  时间序列: {modal_dims['ts']}维 (累计贡献率: {modal_dims['ts_contrib']:.4f})")
    print(f"  向量: {modal_dims['vector']}维 (累计贡献率: {modal_dims['vector_contrib']:.4f})")
    print(f"  PCA降维RF-MSE: {mse:.6f}\n")
    
    return mse, modal_dims


# ----------------------------
# TSVD 基线：遵循“每模态独立标准化→训练拟合→测试复用”，维度用训练集奇异值能量达到 contribution 的最小维度
# ----------------------------
def _tsvd_dim_by_contrib(X_scaled, contribution=0.9):
    # 用训练集的奇异值估计累计方差占比
    s = np.linalg.svd(X_scaled, full_matrices=False, compute_uv=False)
    var = (s**2) / (X_scaled.shape[0] - 1)
    ratio = var / var.sum()
    cum = np.cumsum(ratio)
    dim = int(np.searchsorted(cum, contribution) + 1)
    # TruncatedSVD 需要 n_components < min(n_samples, n_features)
    max_comp = max(1, min(X_scaled.shape[0], X_scaled.shape[1]) - 1)
    dim = max(1, min(dim, max_comp))
    return dim

def train_rf_with_tsvd_contribution(train_data, test_data, contribution=0.9):
    # 时间序列
    scaler_ts = StandardScaler()
    Xts_tr = scaler_ts.fit_transform(train_data.ts_data.reshape(len(train_data), -1))
    dim_ts = _tsvd_dim_by_contrib(Xts_tr, contribution)
    tsvd_ts = TruncatedSVD(n_components=dim_ts, random_state=0).fit(Xts_tr)
    Xts_te = tsvd_ts.transform(scaler_ts.transform(test_data.ts_data.reshape(len(test_data), -1)))
    
    # 向量
    scaler_v = StandardScaler()
    Xv_tr = scaler_v.fit_transform(train_data.vector_data)
    dim_v = _tsvd_dim_by_contrib(Xv_tr, contribution)
    tsvd_v = TruncatedSVD(n_components=dim_v, random_state=0).fit(Xv_tr)
    Xv_te = tsvd_v.transform(scaler_v.transform(test_data.vector_data))
    
    X_train = np.hstack([tsvd_ts.transform(Xts_tr), tsvd_v.transform(Xv_tr)])
    X_test  = np.hstack([Xts_te, Xv_te])
    y_train, y_test = train_data.targets, test_data.targets
    
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse


# ----------------------------
# RP 基线（Gaussian）：每模态维度 = PCA 得到的对应模态维度（公平对齐）
# ----------------------------
def train_rf_with_rp_fixed_dims(train_data, test_data, pca_modal_dims, rp_random_state=0):
    # 时间序列
    scaler_ts = StandardScaler()
    Xts_tr = scaler_ts.fit_transform(train_data.ts_data.reshape(len(train_data), -1))
    Xts_te_src = scaler_ts.transform(test_data.ts_data.reshape(len(test_data), -1))
    dim_ts = int(pca_modal_dims['ts'])
    rp_ts = GaussianRandomProjection(n_components=min(dim_ts, Xts_tr.shape[1]), random_state=rp_random_state).fit(Xts_tr)
    Xts_tr_rp = rp_ts.transform(Xts_tr)
    Xts_te = rp_ts.transform(Xts_te_src)
    
    # 向量
    scaler_v = StandardScaler()
    Xv_tr = scaler_v.fit_transform(train_data.vector_data)
    Xv_te_src = scaler_v.transform(test_data.vector_data)
    dim_v = int(pca_modal_dims['vector'])
    rp_v = GaussianRandomProjection(n_components=min(dim_v, Xv_tr.shape[1]), random_state=rp_random_state).fit(Xv_tr)
    Xv_tr_rp = rp_v.transform(Xv_tr)
    Xv_te = rp_v.transform(Xv_te_src)
    
    X_train = np.hstack([Xts_tr_rp, Xv_tr_rp])
    X_test  = np.hstack([Xts_te, Xv_te])
    y_train, y_test = train_data.targets, test_data.targets
    
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse


# ----------------------------
# 主流程：输出你指定的列（两模态版本）
# 列顺序：客户端ID | 训练样本数 | 测试样本数 | PCA总维度 | 时间序列维度 | 向量维度 | PCA降维RF-MSE | TSVD降维RF-MSE | RP降维RF-MSE | 联邦特征-MSE
# ----------------------------
rows = []
print("===== 两模态：PCA / TSVD / RP（+ OURS 占位） =====")
for client_idx in range(3):
    client_id = client_idx + 1
    train_data = client_train_datasets[client_idx]
    test_data  = client_test_datasets[client_idx]
    
    # 1) PCA（得到各模态维度 + MSE）
    pca_mse, pca_modal_dims = train_rf_with_pca_contribution(
        train_data, test_data, client_id=client_id, contribution=0.9
    )
    # 2) TSVD（训练集奇异值能量 -> 维度）
    tsvd_mse = train_rf_with_tsvd_contribution(train_data, test_data, contribution=0.9)
    # 3) RP（维度与 PCA 对齐）
    rp_mse = train_rf_with_rp_fixed_dims(train_data, test_data, pca_modal_dims=pca_modal_dims, rp_random_state=0)
    
    rows.append({
        '客户端ID': client_id,
        '训练样本数': len(train_data),
        '测试样本数': len(test_data),
        'PCA总维度': int(pca_modal_dims['total']),
        '时间序列维度': int(pca_modal_dims['ts']),
        '向量维度': int(pca_modal_dims['vector']),
        'PCA降维RF-MSE': round(float(pca_mse), 6),
        'TSVD降维RF-MSE': round(float(tsvd_mse), 6),
        'RP降维RF-MSE': round(float(rp_mse), 6),
        '联邦特征-MSE': None  # 后面回填
    })

results_df = pd.DataFrame(rows)

# 回填 OURS（联邦特征）的 MSE（若你已有 results_df1）
try:
    your_method_mses = results_df1['联邦特征-MSE']
    for i in range(len(results_df)):
        results_df.loc[i, '联邦特征-MSE'] = round(float(your_method_mses.iloc[i]), 6)
except Exception:
    pass  # 如果没有，就先留空

print("\n===== 四方法对比（两模态）=====")
print(results_df)


===== 两模态：PCA / TSVD / RP（+ OURS 占位） =====
客户端 1 - PCA后总特征数: 4
  时间序列: 1维 (累计贡献率: 0.9692)
  向量: 3维 (累计贡献率: 1.0000)
  PCA降维RF-MSE: 0.067639

客户端 2 - PCA后总特征数: 4
  时间序列: 1维 (累计贡献率: 0.9809)
  向量: 3维 (累计贡献率: 1.0000)
  PCA降维RF-MSE: 0.008323

客户端 3 - PCA后总特征数: 4
  时间序列: 1维 (累计贡献率: 0.9689)
  向量: 3维 (累计贡献率: 1.0000)
  PCA降维RF-MSE: 0.044063


===== 四方法对比（两模态）=====
   客户端ID  训练样本数  测试样本数  PCA总维度  时间序列维度  向量维度  PCA降维RF-MSE  TSVD降维RF-MSE  \
0      1     21      5       4       1     3     0.067639      0.067131   
1      2     21      5       4       1     3     0.008323      0.039119   
2      3     21      5       4       1     3     0.044063      0.090049   

   RP降维RF-MSE  联邦特征-MSE  
0    0.076459  0.053679  
1    0.037575  0.025789  
2    0.021852  0.036576  
